In [28]:
    from IPython.display import HTML, display

    def set_css():
      display(HTML('''
      <style>
        pre {
          white-space: pre-wrap;
        }
      </style>
      '''))
    get_ipython().events.register('pre_run_cell', set_css)

In [1]:
!pip install datasets

from datasets import load_dataset

dataset = load_dataset("nbertagnolli/counsel-chat")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system ==

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


20220401_counsel_chat.csv:   0%|          | 0.00/4.13M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2775 [00:00<?, ? examples/s]

In [2]:
dataset["train"][0]

{'questionID': 0,
 'questionTitle': 'Do I have too many issues for counseling?',
 'questionText': 'I have so many issues to address. I have a history of sexual abuse, I’m a breast cancer survivor and I am a lifetime insomniac.    I have a long history of depression and I’m beginning to have anxiety. I have low self esteem but I’ve been happily married for almost 35 years.\n   I’ve never had counseling about any of this. Do I have too many issues to address in counseling?',
 'questionLink': 'https://counselchat.com/questions/do-i-have-too-many-issues-for-counseling',
 'topic': 'depression',
 'therapistInfo': 'Jennifer MolinariHypnotherapist & Licensed Counselor',
 'therapistURL': 'https://counselchat.com/therapists/jennifer-molinari',
 'answerText': 'It is very common for\xa0people to have multiple issues that they want to (and need to) address in counseling.\xa0 I have had clients ask that same question and through more exploration, there is often an underlying fear that they\xa0 "can\

In [3]:
def is_valid(example):
    return example["questionText"] and example["answerText"]

cleaned_dataset = dataset["train"].filter(is_valid)

def preprocess(example):
    input_text = "question: " + example["questionText"].strip()
    target_text = example["answerText"].strip()
    return {"input_text": input_text, "target_text": target_text}

processed_dataset = cleaned_dataset.map(preprocess)

print(processed_dataset[0])


Filter:   0%|          | 0/2775 [00:00<?, ? examples/s]

Map:   0%|          | 0/2612 [00:00<?, ? examples/s]

{'questionID': 0, 'questionTitle': 'Do I have too many issues for counseling?', 'questionText': 'I have so many issues to address. I have a history of sexual abuse, I’m a breast cancer survivor and I am a lifetime insomniac.    I have a long history of depression and I’m beginning to have anxiety. I have low self esteem but I’ve been happily married for almost 35 years.\n   I’ve never had counseling about any of this. Do I have too many issues to address in counseling?', 'questionLink': 'https://counselchat.com/questions/do-i-have-too-many-issues-for-counseling', 'topic': 'depression', 'therapistInfo': 'Jennifer MolinariHypnotherapist & Licensed Counselor', 'therapistURL': 'https://counselchat.com/therapists/jennifer-molinari', 'answerText': 'It is very common for\xa0people to have multiple issues that they want to (and need to) address in counseling.\xa0 I have had clients ask that same question and through more exploration, there is often an underlying fear that they\xa0 "can\'t be h

In [4]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
t5_model = T5ForConditionalGeneration.from_pretrained("t5-small").to(device)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [7]:
t5_tokenizer = T5Tokenizer.from_pretrained("t5-small")
t5_model = T5ForConditionalGeneration.from_pretrained("t5-small").to(device)

def tokenize(batch):
    input_encodings = t5_tokenizer(batch["input_text"], padding="max_length", truncation=True, max_length=128)
    target_encodings = t5_tokenizer(batch["target_text"], padding="max_length", truncation=True, max_length=128)

    return {
        "input_ids": input_encodings["input_ids"],
        "attention_mask": input_encodings["attention_mask"],
        "labels": target_encodings["input_ids"]
    }

tokenized_dataset = processed_dataset.map(tokenize, batched=True)

tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

tokenized_dataset[0]


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/2612 [00:00<?, ? examples/s]

{'input_ids': tensor([  822,    10,    27,    43,    78,   186,   807,    12,  1115,     5,
            27,    43,     3,     9,   892,    13,  6949,  5384,     6,    27,
            22,    51,     3,     9,  6748,  1874,   244,  7003,   127,    11,
            27,   183,     3,     9,  6556, 29079,    75,     5,    27,    43,
             3,     9,   307,   892,    13,  7562,    11,    27,    22,    51,
          1849,    12,    43,  6261,     5,    27,    43,   731,  1044,     3,
         19958,    68,    27,    22,   162,   118, 16725,  4464,    21,   966,
          3097,   203,     5,    27,    22,   162,   470,   141, 15457,    81,
           136,    13,    48,     5,   531,    27,    43,   396,   186,   807,
            12,  1115,    16, 15457,    58,     1,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [23]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./t5_therapist_model",
    per_device_train_batch_size=8,
    num_train_epochs=5,
    logging_steps=100
)


In [24]:
from transformers import Trainer

trainer = Trainer(
    model=t5_model,
    args=training_args,
    train_dataset=tokenized_dataset
)

In [25]:
trainer.train()

Step,Training Loss
100,3.139500
200,3.107200
300,3.125800
400,3.125600
500,3.096800
600,3.079500
700,3.151400
800,3.111700
900,3.132500
1000,3.090000


TrainOutput(global_step=1635, training_loss=3.1137402175763333, metrics={'train_runtime': 295.0537, 'train_samples_per_second': 44.263, 'train_steps_per_second': 5.541, 'total_flos': 441890981806080.0, 'train_loss': 3.1137402175763333, 'epoch': 5.0})

In [26]:
def generate_therapist_reply(user_question):
    input_text = "question: " + user_question.strip()
    inputs = t5_tokenizer(input_text, return_tensors="pt", truncation=True, padding=True).to(device)

    output = t5_model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=100,
        num_beams=5,
        no_repeat_ngram_size=2,
        early_stopping=True
    )


    reply = t5_tokenizer.decode(output[0], skip_special_tokens=True)
    return reply


In [29]:
print(generate_therapist_reply("I feel anxious in social situations. What can I do?"))

What if you feel anxious in social situations? It is important to know that your anxiety is a part of your life, and that is what you need to do in the long-term. If you are feeling anxious and anxious, you may be able to deal with it.


In [30]:
print(generate_therapist_reply("I don't feel motivated lately."))
print(generate_therapist_reply("How do I stop comparing myself to others?"))

I'm sorry to hear that you're not feeling motivated and that's a big part of your life. It sounds like you haven't been motivated since then. You've been wanting to know more about what you are aiming for.
I'm sorry that you're not comparing yourself to others. It's a good question to ask yourself if you are able to tell yourself what you want to look for.


In [32]:
t5_model.save_pretrained("t5_therapist_model")
t5_tokenizer.save_pretrained("t5_therapist_model")

!zip -r t5_therapist_model.zip t5_therapist_model

  adding: t5_therapist_model/ (stored 0%)
  adding: t5_therapist_model/checkpoint-981/ (stored 0%)
  adding: t5_therapist_model/checkpoint-981/optimizer.pt (deflated 7%)
  adding: t5_therapist_model/checkpoint-981/config.json (deflated 63%)
  adding: t5_therapist_model/checkpoint-981/trainer_state.json (deflated 79%)
  adding: t5_therapist_model/checkpoint-981/generation_config.json (deflated 29%)
  adding: t5_therapist_model/checkpoint-981/model.safetensors (deflated 9%)
  adding: t5_therapist_model/checkpoint-981/scheduler.pt (deflated 56%)
  adding: t5_therapist_model/checkpoint-981/rng_state.pth (deflated 25%)
  adding: t5_therapist_model/checkpoint-981/training_args.bin (deflated 51%)
  adding: t5_therapist_model/checkpoint-500/ (stored 0%)
  adding: t5_therapist_model/checkpoint-500/optimizer.pt (deflated 7%)
  adding: t5_therapist_model/checkpoint-500/config.json (deflated 63%)
  adding: t5_therapist_model/checkpoint-500/trainer_state.json (deflated 62%)
  adding: t5_therapist_m